In [12]:
import io
from PIL import Image
import os
import wandb
import glob
import torch
import monai
import random
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

%matplotlib inline

### Set-up transforms

In [3]:
torch.manual_seed(1024)
np.random.seed(1024)
device = torch.device(
    "cuda:1"
    if torch.cuda.is_available()
    else
    "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Data Augmentation
transform = transforms.Compose([
    transforms.Resize([256, 256]),
    transforms.RandomRotation(45),
    transforms.RandomResizedCrop([256, 256]),
    transforms.GaussianBlur(3)
])

target_transform = transforms.Compose([
    transforms.Resize([256, 256]),
    transforms.RandomRotation(45),
    transforms.RandomResizedCrop([256, 256])
])

Using cuda:1 device


### Create Segmentation Dataset

In [4]:
class SegDataset(Dataset):
    def __init__(self, data_root, transform, target_transform, train=True):
        self.data_root = data_root
        self.transform = transform
        self.target_transform = target_transform
        self.train = train
        self.gt_files_path = []
        # find all patient directories
        patient_directories = glob.glob(os.path.join(self.data_root, 'patient*'))
        # find all files with the suffix _gt.npy
        train_size = int(len(patient_directories)*0.8)
        
        if self.train:
            for patient_directory in patient_directories[0:train_size]:
                per_patient_file_path = glob.glob(os.path.join(patient_directory, '*_gt.npy'))
                for path in per_patient_file_path:
                    self.gt_files_path.append(path)
        else:
            for patient_directory in patient_directories[train_size:]:
                per_patient_file_path = glob.glob(os.path.join(patient_directory, '*_gt.npy'))
                for path in per_patient_file_path:
                    self.gt_files_path.append(path)
                    
    def __len__(self):
        return len(self.gt_files_path)
    
    def __getitem__(self, index):
        gt_image_path = self.gt_files_path[index]
        image_path = gt_image_path[:-7] + ".npy"
        image = np.load(image_path)
        gt_image = np.load(gt_image_path)
        image = torch.tensor(image[None,:,:]).float()
        gt_image = torch.tensor(gt_image).long()
            
        # Convert the ground truth label to one-hot encoding
        one_hot_label = torch.nn.functional.one_hot(gt_image, num_classes=4)

        # Transpose the tensor to have dimensions (C, H, W)
        one_hot_label = one_hot_label.permute(2, 0, 1)

        # Remove the background channel (dimension 0)
        one_hot_label = one_hot_label[1:, :, :]
        
        # Use seed to make sure image and target has same transform
        seed = np.random.randint(2147483647)
        random.seed(seed)
        torch.manual_seed(seed)
        image = self.transform(image)
        random.seed(seed)
        torch.manual_seed(seed)
        target = self.target_transform(one_hot_label)
        
        return image, target

In [5]:
train_dataset = SegDataset(data_root = './database/training', 
                     transform = transform, 
                     target_transform = target_transform,
                     train = True)

val_dataset = SegDataset(data_root = './database/training', 
                     transform = transform, 
                     target_transform = target_transform,
                     train = False)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

### Init hyperparameters

In [8]:
lr = 1e-4
batch_size = 8
weight_decay = 0
num_epochs = 10

run = wandb.init(
    project="Unet",
    name='test',
    # Track hyperparameters and run metadata
    config={
        "learning rate": lr,
        "batch_size": batch_size,
        "weight decay": weight_decay,
        "Epoches number": num_epochs,
        "transform": str(transform),
        "target transform": str(target_transform)
    })

### Init Model

In [7]:
model = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels
    classes=3,                      # model output channels (number of classes)
)

#preprocess_input = get_preprocessing_fn('resnet50', pretrained='imagenet')

model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)
seg_loss = monai.losses.DiceCELoss(sigmoid=True, squared_pred=True, reduction='mean')

### Train

In [9]:
def vis_img(img, mask):
    # img: (B, 256, 64, 64), {: (B, 1, 256, 256)
    print(f"{img.shape=}, {mask.shape=}")
    img = np.squeeze(img)
    mask = np.squeeze(mask)
    plt.figure()
    plt.imshow(img, 'gray')
    overlay_mask_0 = np.ma.masked_where(mask[0] == 0, img)
    overlay_mask_1 = np.ma.masked_where(mask[1] == 0, img)
    overlay_mask_2 = np.ma.masked_where(mask[2] == 0, img)
    plt.imshow(overlay_mask_0, 'Greens', alpha = 0.7, clim=[0,1], interpolation='nearest')
    plt.imshow(overlay_mask_1, 'Reds', alpha = 0.7, clim=[0,1], interpolation='nearest')
    plt.imshow(overlay_mask_2, 'Purples', alpha = 0.7, clim=[0,1], interpolation='nearest')
    buffer = io.BytesIO()
    plt.savefig(buffer, format='jpeg')
    buffer.seek(0)

    # Convert the in-memory buffer to a NumPy array
    image_array = np.array(Image.open(buffer))
    return image_array

In [ ]:
# train
best_loss = 1e10
val_freq = 5

for epoch in range(num_epochs):
    # Train
    model.train()
    epoch_loss = 0
    for step, (img, gt) in enumerate(tqdm(train_loader)):
        img = img.to(device)
        mask = model(img)
        loss = seg_loss(mask, gt.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    print(f'EPOCH: {epoch + 1}, Train Loss: {epoch_loss}')
    
    # Validation
    if epoch % val_freq == 0:
        model.eval()
        val_loss = 0
        last_image_batch = None
        last_gt_mask_batch = None
        last_pr_mask_batch = None
        with torch.no_grad():
            for step, (img, gt) in enumerate(tqdm(val_loader)):
                img = img.to(device)
                mask = model(img)
                loss = seg_loss(mask, gt.to(device))
                val_loss += loss.item()
                last_image_batch = img
                last_gt_mask_batch = gt
                last_pr_mask_batch = mask
            
    print(f'EPOCH: {epoch + 1}, Validation Loss: {val_loss}')
    
    last_image = last_image_batch.detach().cpu().numpy()[0][0]
    last_gt = last_gt_mask_batch.detach().cpu().numpy()[0]
    last_pr = last_pr_mask_batch.detach().cpu().numpy()[0]
    
    threshold = 0.95  # Set your desired threshold value
    binary_mask = (last_pr > threshold)
    
    ground_truth = vis_img(last_image, last_gt)
    predicted = vis_img(last_image, binary_mask)
    # Log
    wandb.log({"loss": epoch_loss,
               "val_loss": val_loss,
               "ground_truth": wandb.Image(ground_truth),
               "prediction": wandb.Image(predicted)})
    
    # save the best model
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), './model/unet-test/model_best.pth')

100%|██████████| 96/96 [00:55<00:00,  1.72it/s]


EPOCH: 1, Train Loss: 51.06673422455788


100%|██████████| 23/23 [00:08<00:00,  2.82it/s]


EPOCH: 1, Validation Loss: 11.386808454990387
img.shape=(256, 256), mask.shape=(3, 256, 256)
img.shape=(256, 256), mask.shape=(3, 256, 256)


100%|██████████| 96/96 [00:57<00:00,  1.66it/s]


EPOCH: 2, Train Loss: 41.43707627058029
EPOCH: 2, Validation Loss: 11.386808454990387
img.shape=(256, 256), mask.shape=(3, 256, 256)
img.shape=(256, 256), mask.shape=(3, 256, 256)


100%|██████████| 96/96 [00:59<00:00,  1.62it/s]


EPOCH: 3, Train Loss: 35.30691337585449
EPOCH: 3, Validation Loss: 11.386808454990387
img.shape=(256, 256), mask.shape=(3, 256, 256)
img.shape=(256, 256), mask.shape=(3, 256, 256)


 83%|████████▎ | 80/96 [00:48<00:12,  1.33it/s]